# Project 3: Airbnb
**This is the third of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-12-05, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python/PySpark**
- **For each question you may use as many cells for your solution as you like**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**

# Introduction
[Airbnb](http://airbnb.com) is an online marketplace for arranging or offering lodgings. In this project you will use Spark to analyze data obtained from the Airbnb website. The purpose of the analysis is to extract information about trends and patterns from the data.

The project has two parts.

### Part 1: Loading, describing and preparing the data
There's quite a lot of data. Make sure that you can load and correctly parse the data, and that you understand what the dataset contains. You should also prepare the data for the analysis in part two. This means cleaning it and staging it so that subsequent queries are fast.

### Par 2: Analysis
In this part your goal is to learn about trends and usage patterns from the data. You should give solutions to the tasks defined in this notebook, and you should use Spark to do the data processing. You may use other libraries like for instance Pandas and matplotlib for visualisation.

## Guidelines
- Processing data should be done using Spark. Once data has been reduced to aggregate form, you may use collect to extract it into Python for visualisation.
- Your solutions will be evaluated by correctness, code quality and interpretability of the output. This means that you have to write clean and efficient Spark code that will generate sensible execution plans, and that the tables and visualisations that you produce are meaningful and easy to read.
- You may add more cells for your solutions, but you should not modify the notebook otherwise.

### Create Spark session and define imports

In [2]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Project3").getOrCreate()

# Part 1: Loading, describing and preparing the data
The data comes in two files. Start by downloading the files and putting them in your `data/` folder.

- [Listings](https://files.dtu.dk/u/siPzAasj8w2gI_ME/listings.csv?l) (5 GB)
- [Reviews](https://files.dtu.dk/u/k3oaPYp6GjKBeho4/reviews.csv?l) (9.5 GB)

### Load the data
The data has multiline rows (rows that span multiple lines in the file). To correctly parse these you should use the `multiline` option and set the `escape` character to be `"`.

In [3]:
df = spark.read.option('header',True).option('inferSchema',True).option('escape','"').option('multiLine', 'True').csv('samplelistings.csv')

### Describe the data
List the features (schema) and sizes of the datasets.

In [4]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experiences_offered: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- transit: string (nullable = true)
 |-- access: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- house_rules: string (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- medium_url: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- xl_picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: timestamp (nullable = true)
 |-- host_location: string (nu

In [5]:
df.count()

387

### Prepare the data for analysis
You should prepare two dataframes to be used in the analysis part of the project. You should not be concerned with cleaning the data. There's a lot of it, so it will be sufficient to drop rows that have bad values. You may want to go back and refine this step at a later point when doing the analysis.

You may also want to consider if you can stage your data so that subsequent processing is more efficient (this is not strictly necessary for Spark to run, but you may be able to decrease the time you sit around waiting for Spark to finish things)

In [6]:
df = df.filter(f.col('city').isNotNull() & f.col('price').isNotNull())

# Part 2: Analysis
Use Spark and your favorite tool for data visualization to solve the following tasks.

## The basics
Compute and show a dataframe with the number of listings and neighbourhoods per city.

In [7]:
df.groupBy('city').agg(f.countDistinct('id'),f.countDistinct('neighbourhood')).show()

+--------------------+------------------+-----------------------------+
|                city|count(DISTINCT id)|count(DISTINCT neighbourhood)|
+--------------------+------------------+-----------------------------+
|         JOHANNESHOV|                 1|                            1|
|            Hagsätra|                 1|                            1|
|Enskede-Årsta-Vantör|                 1|                            1|
|         Kungsholmen|                 2|                            1|
|           Stockholm|               352|                           14|
|     Skarpnäcks gård|                 2|                            1|
|           skarpnäck|                 1|                            1|
|           BANDHAGEN|                 1|                            1|
|           HÄGERSTEN|                 1|                            1|
|             Långbro|                 1|                            1|
|            Norrmalm|                 2|                       

In [12]:
df.select(f.col('review_scores_rating')).sort(f.desc('review_scores_rating')).show(1000, False)

+--------------------+
|review_scores_rating|
+--------------------+
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100                 |
|100       

## Prices
Compute the minimum, maximum and average listing price in your city. 

In [9]:
stockholm_df = df.filter(f.col('city')=='Stockholm')
stockholm_df.agg(f.min('price'),f.max('price'),f.avg('price')).show()

+----------+----------+----------+
|min(price)|max(price)|avg(price)|
+----------+----------+----------+
| $1,008.00|   $998.00|      null|
+----------+----------+----------+



Compute and visualize the distribution of listing prices in your city.

The value of a listing is its rating divided by its price.

Compute and show a dataframe with the 3 highest valued listings in each neighbourhood.

In [16]:
stockholm_df = stockholm_df.filter(f.col('review_scores_rating').isNotNull())
stockholm_df = stockholm_df.withColumn('Value', f.col('review_scores_rating')/f.col('price'))
sorted_by_value_window = Window.partitionBy('neighbourhood').orderBy(f.desc('Value'))
ranked_df = stockholm_df.withColumn('NeighbourhoodValueRank', f.rank().over(sorted_by_value_window))
ranked_df.filter(f.col('NeighbourhoodValueRank') <= 3).select('city','neighbourhood','NeighbourhoodValueRank','review_scores_rating','price').orderBy('neighbourhood', f.desc('Value')).show()

+---------+--------------------+----------------------+--------------------+---------+
|     city|       neighbourhood|NeighbourhoodValueRank|review_scores_rating|    price|
+---------+--------------------+----------------------+--------------------+---------+
|Stockholm|              Bromma|                     1|                 100|$1,996.00|
|Stockholm|              Bromma|                     1|                  96|$1,291.00|
|Stockholm|              Bromma|                     1|                  91|  $382.00|
|Stockholm|              Bromma|                     1|                  98|$1,947.00|
|Stockholm|              Bromma|                     1|                 100|  $900.00|
|Stockholm|              Bromma|                     1|                  92|$1,761.00|
|Stockholm|              Bromma|                     1|                  81|  $450.00|
|Stockholm|              Bromma|                     1|                 100|  $753.00|
|Stockholm|              Bromma|           

## Trends
Now we want to analyze the "popularity" of your city. The data does not contain the number of bookings per listing, but we have a large number of reviews, and we will assume that this is a good indicator of activity on listings.

Compute and visualize the popularity (i.e., number of reviews) of your city over time.

Compute and visualize the popularity of neighbourhoods over time. If there are many neighbourhoods in your city, you should select a few interesting ones for comparison.

Compute and visualize the popularity of your city by season. For example, visualize the popularity of your city per month.

## Reviews
In this part you should determine which words used in reviews that are the most positive. 

The individual reviews do not have a rating of the listing, so we will assume that each review gave the average rating to the listing, i.e., the one on the listing.

You should assign a positivity weight to each word seen in reviews and list the words with the highest weight. It is up to you to decide what the weight should be. For example, it can be a function of the rating on the listing on which it occurs, the number of reviews it occurs in, and the number of unique listings for which it was used to review.

Depending on your choice of weight function, you may also want to do some filtering of words. For example, remove words that only occur in a few reviews.